# Crawling

Crawling adalah proses mengambil atau menyalin data dari suatu sumber di internet secara otomatis menggunakan program atau skrip. Di sini saya akan mencoba crawling data pada jurnal springer nature https://www.springernature.com/gp/librarians/products/journals/nature-research-journals/nature-reviews. Berikut adalah proses-prosesnya:

## Import Library yang Dibutuhkan

In [16]:
pip install sprynger

In [18]:
import requests
import csv
import time


## Menyiapkan Kunci API (API Key)
Pertama, kita perlu mendapatkan API Key dari Springer Nature Developer Portal. Kunci ini berfungsi sebagai identitas agar server Springer mengenali siapa yang mengakses data. Tanpa API Key, permintaan data tidak akan dilayani. API Key dimasukkan ke dalam variabel api_key.

## Menentukan Kata Kunci Pencarian

Proses berikutnya adalah menentukan kata kunci yang ingin kita cari, misalnya di sini saya menentukan kata kunci "web mining", "web usage mining".
Crawler akan menggunakan kata kunci ini untuk mencari artikel yang sesuai di database Springer.

In [19]:
api_key = "34cf8373948405e957dffcbdda56bca5"  # ganti dengan API key Anda
keywords = ["web mining", "web usage mining", "text mining"]
url = "https://api.springernature.com/meta/v2/json"
output_file = "springer_results.csv"


Menyiapkan file CSV kosong lalu tulis header kolom: keyword, doi, title, abstract.

In [20]:
f = open(output_file, "w", newline="", encoding="utf-8-sig")
writer = csv.writer(f)
writer.writerow(["keyword", "doi", "title", "abstract"])


28

## Mengirim Permintaan ke API
Setelah menentukam kata kunci, selanjutnya mengirimkan permintaan (request) ke API Springer.
Permintaan ini akan membawa informasi:

kata kunci yang kita cari (q)

API Key sebagai identitas

jumlah hasil yang ingin kita ambil (p)

Jika permintaan berhasil, API akan mengembalikan data dalam format JSON.

## Mengecek dan Mengambil Data

Ketika API merespons, kemudian cek apakah permintaan berhasil.
Jika berhasil (status_code = 200), maka data JSON yang berisi artikel akan diambil.
Dari data tersebut, kita bisa menyaring informasi penting seperti DOI, judul, dan abstrak artikel.

## Menyimpan Hasil ke CSV

Langkah terakhir adalah menyimpan hasil crawling ke dalam file CSV.
Kita membuat file bernama springer_results.csv yang berisi:

- kata kunci pencarian

- DOI

- judul artikel

- abstrak artikel

In [21]:
for kw in keywords:
    params = {
        "q": kw,
        "api_key": api_key,
        "p": 10  # jumlah hasil per query (maks 50)
    }
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        print(f"\nKata kunci: {kw}")
        print(f"Total hasil: {data['result'][0]['total']}\n")

        for record in data.get("records", []):
            doi = record.get("doi", "N/A")
            title = record.get("title", "No title")
            abstract = record.get("abstract", "No abstract")

            # Tulis ke CSV
            writer.writerow([kw, doi, title, abstract])

            # Print ringkasan ke console
            print(f"DOI: {doi}")
            print(f"Title: {title}")
            print(f"Abstract: {abstract[:200]}...\n")
    else:
        print("Error:", response.status_code, response.text)

    # jeda antar request agar tidak dianggap spam
    time.sleep(1)



Kata kunci: web mining
Total hasil: 317019

DOI: 10.1007/978-3-032-00983-8_5
Title: Survey on Data Mining and Machine Learning Methods Used in Analyzing Tweets
Abstract: It is observed that the Mental illness by the actions and individual emotions and expressions towards a particular situation. It indicates that American Psychiatric Association that has 19% of people ...

DOI: 10.1007/978-3-031-93802-3_7
Title: Unveiling Power Laws in Graph Mining: Techniques and Applications in Graph Query Analysis
Abstract: Power laws play a crucial role in understanding the structural and functional properties of real-world graphs, influencing various aspects of graph mining and query processing. This paper explores the...

DOI: 10.1007/978-981-96-7238-7_2
Title: Architecture Mining Approach for Systems-of-Systems: Monitoring and Discovery
Abstract: Context: Systems of Systems (SoS) constitute a type of complex software systems resulting from integrating heterogeneous constituent systems that are i

In [22]:
f.close()
print("Selesai. Hasil disimpan di:", output_file)


Selesai. Hasil disimpan di: springer_results.csv
